In [155]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time

In [156]:
driver = webdriver.Chrome('/home/lucas.cardoso/fastfinger-tensorflow/chromedriver_linux64/chromedriver')
driver.get('https://10fastfingers.com/login')

In [1]:
login = input("Email: ")
password = input ("Password: ")

KeyboardInterrupt: 

In [ ]:
driver.find_element('id', 'UserEmail').send_keys(login)
driver.find_element('id', 'UserPassword').send_keys(password)
driver.find_element('id', 'login-form-submit').click()

In [ ]:
while True:
    try:
        highlightWord = driver.find_elements_by_class_name("highlight")[0]   
    except:
        break
    driver.find_element('id','inputfield').send_keys(highlightWord.text + " ")
    time.sleep(0.07)

In [ ]:
driver.get("https://10fastfingers.com/anticheat/view/1/1")
driver.find_element('id', 'start-btn').click()

In [ ]:
imageUrl = driver.find_elements_by_tag_name('img')[0];
driver.save_screenshot("screenshot.png")

In [ ]:
from PIL import Image
imageObject  = Image.open("screenshot.png")
cropped = imageObject.crop((300,190,880,320))
cropped.save("screenshot.png")